 # Import necessary libraries 

In [32]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from datetime import datetime 
from datetime import timezone
import time
from sklearn import preprocessing

import torch
import torch.nn as nn
from torch.optim import Adam
import torch.utils.data as data_utils 
import cryptocompare
from IPython.display import Image

# Define and intialize the neural network

In [33]:
class NeuralNet(nn.Module):
    def __init__(self, 
                 activation_function = nn.ReLU,
                 optimizer = Adam,
                 dropout = 0.2
                ):
        super(NeuralNet, self).__init__()
        input_dim = 14
        hidden_dim = 100
        output_dim = 1
        self.lstm1 = nn.LSTM(input_dim, hidden_dim)
        self.dropout1 = nn.Dropout(p = dropout)
        self.lstm2 = nn.LSTM(hidden_dim, hidden_dim)
        self.layers =[nn.Dropout(p = dropout)]
        self.layers.append(nn.Linear(hidden_dim, output_dim))
        self.layers = nn.Sequential(*self.layers)
        


    def forward(self, x):
        out, _ = self.lstm1(x.view(len(x), 1 , -1))
        out = self.dropout1(out.view(len(x), -1))
        out, _ = self.lstm2(out.view(len(x), 1 , -1))
        out = self.layers(out.view(len(x), -1))
        
        return out

# DATA

In [34]:
data = cryptocompare.get_historical_price_day('BTC', 'USD', limit=2000, exchange='CCCAGG')
datetime.timestamp(datetime(2021,6,14))

1623621600.0

In [35]:
# get Data api crypto compare 
def get_data(start_date,end_date,crypto):
    l=[]
    i=0

    while start_date>=end_date:
        i=i+1
        print("requette :",i)
        try : 
            data = cryptocompare.get_historical_price_day(crypto, 'USD', limit=2000, exchange='CCCAGG', toTs=start_date)
            if data is None : 
                raise ValueError 
        except: 
            break

        l=data+l
        start_date=datetime.fromtimestamp(data[0]['time'])
    df = pd.DataFrame.from_dict(l)
    df=df.iloc[:,:7]
    return df

In [36]:
#const
START_DATE=datetime.now()
END_DATE=datetime(2010,6,18)
CRYPTO='ETH'
#PATHz='/home/jbouhadoun/esgi/PA5_Crypto-Advice/src/scrapper/outputs/ltc_2018-01-01_2021-06-25.csv'
PATH_BTC="/home/jbouhadoun/Téléchargements/histo_eth (1).csv"
PATH_BTC_TWEET="/home/jbouhadoun/esgi/exp/final_data/historical_eth_twitter_withsentiment.csv"
PATH_MODEL_BTC=""
PATH_scaler_BTC=""

In [37]:
def read_csv(PATH):
    df=pd.read_csv(PATH)
    df=df.fillna(df.mean())
    df=df.iloc[:,2:]
    return df

In [38]:
def read_csv_tweet(PATH):
    df_tweet=pd.read_csv(PATH)
    df_tweet=df_tweet.groupby("time",as_index=False)["sentiment_analysis"].mean()
    df_tweet['time'] = pd.to_datetime(df_tweet['time'], format="%Y-%m-%d")
    df_tweet['time'] = df_tweet['time'].apply(lambda x: int(x.replace(tzinfo=timezone.utc).timestamp()))
    return df_tweet

In [39]:
T=read_csv_tweet(PATH_BTC_TWEET)
#!!!!warning
#T.to_csv("/home/jbouhadoun/esgi/exp/final_data/historical_tweet_eth_final_timestamp.csv")
#T["Text"]=T["Text"].replace(r"\n{2,}","\n")
#T["Embtext"]=T["Embtext"].replace(r"\n{2,}","\n")
#T=T.drop(columns=["Unnamed: 0"])
#T.to_csv("/home/jbouhadoun/esgi/exp/final_data/historical_tweet_eth_final_timestamp_chariots.csv",index=False)
T

,time,sentiment_analysis
0,1483228800,0.120282
1,1483315200,0.054745
2,1483401600,0.127372
3,1483488000,0.063388
4,1483574400,0.096465
...,...,...
1448,1624492800,0.129996
1449,1624579200,0.121893
1450,1624665600,0.106218
1451,1624752000,0.125983


In [40]:
def merge_data(df,df1,df2):
    df_merge=pd.merge(df,df1,on="time")
    df_merge_2=pd.merge(df_merge,df2,on='time')
    df_merge=df_merge_2
    df_merge["time"]=pd.to_datetime(df_merge["time"], unit='s')
    df_merge = df_merge.set_index('time')
    df_merge.index = pd.to_datetime(df_merge.index, unit='s')
    return df_merge

In [41]:
def split_data(data, train_prop=100):
    train_data = data.iloc[:int(train_prop * len(data))]
    test_data = data.iloc[int(train_prop * len(data)):]
    return train_data, test_data

In [42]:
def normalize_data(traindf,min_max_scaler=None):
    x = traindf.values
    if min_max_scaler is None:
        min_max_scaler = preprocessing.MinMaxScaler()
        min_max_scaler.fit(x)
    x_scaled=min_max_scaler.transform(traindf.values)
    normalized_df = pd.DataFrame(x_scaled, columns = traindf.columns)
    return normalized_df,min_max_scaler

In [43]:
def denormalize_data(min_max_scaler, normalized_df):
    new_x = normalized_df.values* (min_max_scaler.data_max_-min_max_scaler.data_min_) + min_max_scaler.data_min_
    return pd.DataFrame(new_x, columns = df.columns)

In [44]:
def extract_window_data(df, window_len=5, zero_base=True):
    window_data = []
    for idx in range(len(df) - window_len):
        tmp = df[idx: (idx + window_len)].copy()
        if zero_base:
            tmp = normalise_zero_base(tmp)
        window_data.append(tmp.values)
    return np.array(window_data)

In [45]:
def plot_curves(train_data,test_data,to_plot, labels, title='', x_label='', y_label=''):
    fig, ax = plt.subplots(1, figsize=(14, 7))
    for i in range(len(to_plot)):
        ax.plot(to_plot[i], label=labels[i])
    
    ax.set_xlabel(x_label)
    ax.set_ylabel(y_label)
    ax.set_title(title)
    ax.legend()
############plot_curves([train_data['close'], test_data['close']], ['training', 'test'], title='btc')

In [46]:
def create_data_loader(train_normalized):
    train_target_tensor = torch.tensor(train_normalized['close'].values.astype(np.float64))
    train_data_tensor = torch.tensor(train_normalized.drop('close', axis = 1).values) 
    train_tensor = data_utils.TensorDataset(train_data_tensor, train_target_tensor) 
    train_loader = data_utils.DataLoader(dataset = train_tensor, batch_size = 32, shuffle = True)
    return train_loader


def create_data_loader_test(test_data,scaler):
    test_normalized,_ = normalize_data(test_data,scaler)
    test_target = torch.tensor(test_normalized['close'].values.astype(np.float64))
    test = torch.tensor(test_normalized.drop('close', axis = 1).values) 
    print(test_target[0])
    test_tensor = data_utils.TensorDataset(test, test_target) 
    test_loader = data_utils.DataLoader(dataset = test_tensor, batch_size = 32, shuffle = False)
    return test_loader



In [47]:
from tqdm import tqdm

def model(train_loader,epochs=1000):
    model = NeuralNet()

    EPOCHS = epochs

    #target = pd.DataFrame(data = train_normalized['close'])


    criterion = nn.MSELoss()
    optimizer = Adam(model.parameters(), lr=0.0001)

    losses = []
    for epoch in range(EPOCHS):
        epoch_loss = 0
        for bidx, batch in tqdm(enumerate(train_loader)):
            X, Y = batch
            model.zero_grad()
            loss = 0
            for i in range(len(X)):
                x, y = X[i], Y[i]
                x = x.view(-1, len(x))

                #Forward Pass
                y_hat = model(x.float())

                #Loss
                loss += criterion(y.float(), y_hat.float())

            #Backward pass
            loss.backward()

            #Parameters optimization
            optimizer.step()

            epoch_loss += float(loss)

        losses.append(epoch_loss)
        model_close=model
        print("Epoch ", epoch, ": ", epoch_loss)
    return model

In [48]:
def predict(model,test_loader,scaler):
    y_list = []
    y_hat_list = []
    min_max_scaler=scaler
    for bidx, batch in tqdm(enumerate(test_loader)):
            X, Y = batch
            for i in range(len(X)):
                x, y = X[i], Y[i]
                x = x.view(-1, len(x))
                y_hat = model(x.float())
                
                print(y_hat)
                vmin = min_max_scaler.data_min_[5]
                vmax = min_max_scaler.data_max_[5]
                y_hat_list.append(float(y_hat)* (vmax - vmin) + vmin)
    return y_hat_list

In [49]:
df=get_data(START_DATE,END_DATE,CRYPTO)
df1=read_csv(PATH_BTC)

df2=read_csv_tweet(PATH_BTC_TWEET)
df_merge=merge_data(df,df1,df2)
#print(df_merge.columns)
#df_merge=df_merge.drop(columns=["coin"])
train_data,test_data=split_data(df_merge,0.99)
train_normalized,scaler = normalize_data(train_data)
train_loader=create_data_loader(train_normalized)
test_loader=create_data_loader_test(test_data,scaler)

df_merge
#df_merge.to_csv("/home/jbouhadoun/esgi/exp/juscrap/outputs/train_btc.csv")
df_merge

requette : 1
requette : 2
requette : 3
tensor(0.5663, dtype=torch.float64)


,high,low,open,volumefrom,volumeto,close,addresses_active_count,addresses_new_non_zero_count,addresses_count,addresses_receiving_count,addresses_sending_count,transactions_transfers_volume_sum,mining_hash_rate_mean,mining_difficulty_latest,sentiment_analysis
time,,,,,,,,,,,,,,,
2017-01-01,8.891,7.970,8.018,854000.05,6.963217e+06,8.154,13695,1357,729071,12933,6215,1.818131e+06,5.646216e+12,8.174688e+13,0.120282
2017-01-02,8.568,7.964,8.154,859902.07,7.152179e+06,8.317,14916,1901,730972,13899,7320,2.857197e+06,5.646488e+12,8.278692e+13,0.054745
2017-01-03,9.940,8.185,8.335,571218.00,5.292232e+06,9.595,14612,2518,733490,13201,8236,5.066915e+06,5.623616e+12,8.197531e+13,0.127372
2017-01-04,11.500,10.240,9.639,2063444.32,2.269524e+07,11.000,17688,3245,736735,15979,10535,7.864367e+06,6.200869e+12,8.832221e+13,0.063388
2017-01-05,10.370,9.151,11.000,2134380.99,2.160218e+07,10.120,18556,3123,739858,16927,9962,8.078451e+06,6.344418e+12,9.108022e+13,0.096465
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-06-22,1998.750,1705.780,1887.890,1703182.02,3.161540e+09,1880.830,516669,91165,120604388,268881,247278,5.269369e+06,4.889401e+14,6.623077e+15,0.105962
2021-06-23,2044.910,1827.200,1880.830,714905.63,1.416402e+09,1968.560,512525,88383,120692771,246050,251778,2.275053e+06,4.735033e+14,6.530681e+15,0.072830
2021-06-24,2034.570,1886.170,1968.560,448512.52,8.825040e+08,1989.010,523980,92943,120785714,239936,270342,2.544816e+06,4.731043e+14,6.420884e+15,0.129996




# WITH WINDOW

In [50]:
df_merge

,high,low,open,volumefrom,volumeto,close,addresses_active_count,addresses_new_non_zero_count,addresses_count,addresses_receiving_count,addresses_sending_count,transactions_transfers_volume_sum,mining_hash_rate_mean,mining_difficulty_latest,sentiment_analysis
time,,,,,,,,,,,,,,,
2017-01-01,8.891,7.970,8.018,854000.05,6.963217e+06,8.154,13695,1357,729071,12933,6215,1.818131e+06,5.646216e+12,8.174688e+13,0.120282
2017-01-02,8.568,7.964,8.154,859902.07,7.152179e+06,8.317,14916,1901,730972,13899,7320,2.857197e+06,5.646488e+12,8.278692e+13,0.054745
2017-01-03,9.940,8.185,8.335,571218.00,5.292232e+06,9.595,14612,2518,733490,13201,8236,5.066915e+06,5.623616e+12,8.197531e+13,0.127372
2017-01-04,11.500,10.240,9.639,2063444.32,2.269524e+07,11.000,17688,3245,736735,15979,10535,7.864367e+06,6.200869e+12,8.832221e+13,0.063388
2017-01-05,10.370,9.151,11.000,2134380.99,2.160218e+07,10.120,18556,3123,739858,16927,9962,8.078451e+06,6.344418e+12,9.108022e+13,0.096465
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-06-22,1998.750,1705.780,1887.890,1703182.02,3.161540e+09,1880.830,516669,91165,120604388,268881,247278,5.269369e+06,4.889401e+14,6.623077e+15,0.105962
2021-06-23,2044.910,1827.200,1880.830,714905.63,1.416402e+09,1968.560,512525,88383,120692771,246050,251778,2.275053e+06,4.735033e+14,6.530681e+15,0.072830
2021-06-24,2034.570,1886.170,1968.560,448512.52,8.825040e+08,1989.010,523980,92943,120785714,239936,270342,2.544816e+06,4.731043e+14,6.420884e+15,0.129996


In [51]:
df_merge.columns

Index(['high', 'low', 'open', 'volumefrom', 'volumeto', 'close',
       'addresses_active_count', 'addresses_new_non_zero_count',
       'addresses_count', 'addresses_receiving_count',
       'addresses_sending_count', 'transactions_transfers_volume_sum',
       'mining_hash_rate_mean', 'mining_difficulty_latest',
       'sentiment_analysis'],
      dtype='object')

In [52]:
all_data=df_merge
test_data_size = 100

train_data = all_data[:-test_data_size]
test_data = all_data[-test_data_size:]

In [53]:
print(len(train_data))
print(len(test_data))

1351
100


In [54]:
print(test_data)

               high      low     open  volumefrom      volumeto    close  \
time                                                                       
2021-03-19  1840.66  1734.36  1776.16   324695.13  5.846494e+08  1809.76   
2021-03-20  1868.98  1800.95  1809.76   272549.21  5.002625e+08  1805.49   
2021-03-21  1817.33  1747.94  1805.49   343447.52  6.123741e+08  1783.94   
2021-03-22  1807.19  1657.58  1783.94   496567.29  8.612933e+08  1682.05   
2021-03-23  1721.69  1652.19  1682.05   392868.19  6.646061e+08  1668.69   
...             ...      ...      ...         ...           ...      ...   
2021-06-22  1998.75  1705.78  1887.89  1703182.02  3.161540e+09  1880.83   
2021-06-23  2044.91  1827.20  1880.83   714905.63  1.416402e+09  1968.56   
2021-06-24  2034.57  1886.17  1968.56   448512.52  8.825040e+08  1989.01   
2021-06-25  2019.45  1791.99  1989.01   882983.22  1.653238e+09  1810.27   
2021-06-26  1851.36  1717.91  1810.27   575329.88  1.021388e+09  1830.94   

           

In [57]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler(feature_range=(-1, 1))
train_data_normalized = scaler.fit_transform(train_data)

In [58]:
print(train_data_normalized[:5])

[[-0.99968177 -0.99999363 -1.         -0.80950878 -0.99711792 -1.
  -1.         -1.         -1.         -1.         -1.         -0.94053307
  -0.99989348 -1.         -0.21878073]
 [-1.         -1.         -0.99986047 -0.80795637 -0.99701925 -0.99983276
  -0.99651407 -0.99686525 -0.99996405 -0.99668418 -0.9945514  -0.89539484
  -0.9998922  -0.99964005 -0.61055071]
 [-0.99864824 -0.99976531 -0.99967478 -0.88388881 -0.99799037 -0.99852153
  -0.99738198 -0.99330984 -0.99991642 -0.99908008 -0.99003474 -0.79940215
  -1.         -0.99992094 -0.17639953]
 [-0.99711126 -0.99758304 -0.99833697 -0.49138921 -0.98890394 -0.99708
  -0.98860006 -0.98912057 -0.99985505 -0.98954454 -0.9786987  -0.67787761
  -0.99727926 -0.99772435 -0.558884  ]
 [-0.99822459 -0.99873948 -0.99694068 -0.47273077 -0.98947464 -0.99798288
  -0.98612194 -0.98982358 -0.99979598 -0.98629051 -0.98152408 -0.66857759
  -0.99660268 -0.99676984 -0.36115415]]


In [59]:
train_data_normalized = torch.FloatTensor(train_data_normalized)

In [60]:
train_window = 12
train_data_normalized.shape

torch.Size([1351, 15])

In [61]:
def create_inout_sequences(input_data, tw=10):
    inout_seq = []
    L = len(input_data)
    for i in range(L-tw):
        train_seq = input_data[i:i+tw]
        train_label = input_data[i+tw:i+tw+1]
        inout_seq.append((train_seq ,train_label))
    return inout_seq

In [62]:
train_inout_seq = create_inout_sequences(train_data_normalized, train_window)

In [64]:
train_inout_seq[:5]

[(tensor([[-0.9997, -1.0000, -1.0000, -0.8095, -0.9971, -1.0000, -1.0000, -1.0000,
           -1.0000, -1.0000, -1.0000, -0.9405, -0.9999, -1.0000, -0.2188],
          [-1.0000, -1.0000, -0.9999, -0.8080, -0.9970, -0.9998, -0.9965, -0.9969,
           -1.0000, -0.9967, -0.9946, -0.8954, -0.9999, -0.9996, -0.6106],
          [-0.9986, -0.9998, -0.9997, -0.8839, -0.9980, -0.9985, -0.9974, -0.9933,
           -0.9999, -0.9991, -0.9900, -0.7994, -1.0000, -0.9999, -0.1764],
          [-0.9971, -0.9976, -0.9983, -0.4914, -0.9889, -0.9971, -0.9886, -0.9891,
           -0.9999, -0.9895, -0.9787, -0.6779, -0.9973, -0.9977, -0.5589],
          [-0.9982, -0.9987, -0.9969, -0.4727, -0.9895, -0.9980, -0.9861, -0.9898,
           -0.9998, -0.9863, -0.9815, -0.6686, -0.9966, -0.9968, -0.3612],
          [-0.9983, -0.9990, -0.9978, -0.5325, -0.9907, -0.9980, -0.9911, -0.9917,
           -0.9997, -0.9926, -0.9819, -0.6973, -0.9964, -0.9973, -0.3534],
          [-0.9981, -0.9987, -0.9979, -0.6891, -0.99

In [65]:
class LSTM(nn.Module):
    def __init__(self, input_size=15, hidden_layer_size=100, output_size=15):
        super().__init__()
        self.hidden_layer_size = hidden_layer_size

        self.lstm = nn.LSTM(input_size, hidden_layer_size)

        self.linear = nn.Linear(hidden_layer_size, output_size)

        self.hidden_cell = (torch.zeros(1,1,self.hidden_layer_size),
                            torch.zeros(1,1,self.hidden_layer_size))

    def forward(self, input_seq):
        lstm_out, self.hidden_cell = self.lstm(input_seq.view(len(input_seq) ,1, -1), self.hidden_cell)
        predictions = self.linear(lstm_out.view(len(input_seq), -1))
        return predictions[-1]


In [66]:
model = LSTM()
loss_function = nn.L1Loss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)


In [67]:
print(model)

LSTM(
  (lstm): LSTM(15, 100)
  (linear): Linear(in_features=100, out_features=15, bias=True)
)


In [ ]:
epochs = 500

for i in range(epochs):
    for seq, labels in train_inout_seq:
        optimizer.zero_grad()
        model.hidden_cell = (torch.zeros(1, 1, model.hidden_layer_size),
                        torch.zeros(1, 1, model.hidden_layer_size))

        y_pred = model(seq)
        #print(y_pred)

        single_loss = loss_function(y_pred, labels)
        single_loss.backward()
        optimizer.step()

    if i%10 == 1:
        print(f'epoch: {i:3} loss: {single_loss.item():10.8f}')

print(f'epoch: {i:3} loss: {single_loss.item():10.10f}')


/home/jbouhadoun/anaconda3/envs/nlp/lib/python3.8/site-packages/torch/nn/modules/loss.py:96: UserWarning: Using a target size (torch.Size([1, 15])) that is different to the input size (torch.Size([15])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.l1_loss(input, target, reduction=self.reduction)
/home/jbouhadoun/anaconda3/envs/nlp/lib/python3.8/site-packages/torch/autograd/__init__.py:145: UserWarning: CUDA initialization: The NVIDIA driver on your system is too old (found version 10010). Please update your GPU driver by downloading and installing a new version from the URL: http://www.nvidia.com/Download/index.aspx Alternatively, go to: https://pytorch.org to install a PyTorch version that has been compiled with your version of the CUDA driver. (Triggered internally at  /opt/conda/conda-bld/pytorch_1616554788289/work/c10/cuda/CUDAFunctions.cpp:109.)
  Variable._execution_engine.run_backward(


epoch:   1 loss: 0.08347397
epoch:  11 loss: 0.05341026
epoch:  21 loss: 0.05265794
epoch:  31 loss: 0.05020534
epoch:  41 loss: 0.05856584
epoch:  51 loss: 0.05853255
epoch:  61 loss: 0.07900714
epoch:  71 loss: 0.04879310
epoch:  81 loss: 0.04955810
epoch:  91 loss: 0.03649808
epoch: 101 loss: 0.06482140
epoch: 111 loss: 0.03799288
epoch: 121 loss: 0.03644188
epoch: 131 loss: 0.04515873
epoch: 141 loss: 0.03689824
epoch: 151 loss: 0.04851750
epoch: 161 loss: 0.02754503
epoch: 171 loss: 0.03241931


In [103]:
fut_pred = 10
test_inputs = train_data_normalized[-train_window:].tolist()

In [104]:

model.eval()
predictions = []
for i in range(fut_pred):
    seq = torch.FloatTensor(test_inputs[i:i+train_window])
    with torch.no_grad():
        model.hidden = (torch.zeros(1, 1, model.hidden_layer_size),
                        torch.zeros(1, 1, model.hidden_layer_size))
        
        predictions.append(model(seq).tolist())


In [105]:
actual_predictions = scaler.inverse_transform(np.array(predictions ))
print(actual_predictions)

[[1.83265881e+03 1.70192218e+03 1.74849096e+03 7.88735084e+05
  1.54628855e+09 1.81396625e+03 1.13804474e+06 5.40361770e+05
  7.92716278e+08 8.02270682e+05 6.40036491e+05 2.74496746e+06
  1.51866428e+20 9.17275343e+22 5.87275595e-02]
 [1.84166964e+03 1.67760299e+03 1.75171425e+03 9.26399625e+05
  1.72120530e+09 1.80096769e+03 1.14747083e+06 5.46937061e+05
  7.94038119e+08 8.13268872e+05 6.44337062e+05 2.77730290e+06
  1.53480354e+20 9.15909167e+22 5.83695003e-02]
 [1.85023884e+03 1.64073697e+03 1.75289201e+03 9.76420711e+05
  1.92396399e+09 1.78293515e+03 1.17725919e+06 5.70925416e+05
  7.96454720e+08 8.44476110e+05 6.56323578e+05 2.74722048e+06
  1.56316640e+20 9.24767394e+22 5.51282651e-02]
 [1.87121930e+03 1.65983633e+03 1.78166201e+03 8.98094591e+05
  1.79733333e+09 1.79940019e+03 1.13302728e+06 5.35060421e+05
  8.01892620e+08 7.97614269e+05 6.35132606e+05 2.68881149e+06
  1.54825935e+20 9.20053674e+22 5.83256071e-02]
 [1.87313743e+03 1.65039644e+03 1.78425377e+03 9.38111071e+05
  

In [108]:
actual_predictions[:,5]

array([1813.96625248, 1800.96768779, 1782.93514934, 1799.4001916 ,
       1790.93053017, 1804.59067098, 1821.2593913 , 1815.9024141 ,
       1862.24957352, 1803.53405469])

In [109]:
test_data[12:23]

,high,low,open,volumefrom,volumeto,close,addresses_active_count,addresses_new_non_zero_count,addresses_count,addresses_receiving_count,addresses_sending_count,transactions_transfers_volume_sum,mining_hash_rate_mean,mining_difficulty_latest,sentiment_analysis
time,,,,,,,,,,,,,,,
2021-03-24,1740.30,1546.93,1668.69,773811.51,1.282635e+09,1583.24,1157951,529959,804691617,780451,699861,1.641230e+06,1.646360e+20,9.391186e+22,0.117733
2021-03-25,1621.59,1549.08,1583.24,532202.27,8.454033e+08,1586.98,1206997,562859,805254476,847846,732501,1.713647e+06,1.686305e+20,9.391186e+22,0.108973
2021-03-26,1700.19,1586.80,1586.98,398023.77,6.530069e+08,1699.89,1094369,535173,805789649,790536,634876,1.455012e+06,1.473042e+20,9.391186e+22,0.095492
2021-03-27,1732.36,1667.18,1699.89,240514.86,4.094073e+08,1713.94,1149816,490264,806279913,722981,733588,1.233196e+06,1.722160e+20,9.391186e+22,0.118015
2021-03-28,1725.59,1662.16,1713.94,203653.93,3.456104e+08,1686.74,1070209,430727,806710640,629178,713202,9.208197e+05,1.729981e+20,9.391186e+22,0.066098
2021-03-29,1840.88,1676.68,1686.74,485643.56,8.617188e+08,1816.48,1116526,524470,807235110,776748,655259,1.607366e+06,1.648553e+20,9.391186e+22,0.011255
2021-03-30,1859.58,1786.81,1816.48,321661.61,5.884028e+08,1841.40,1150659,557303,807792413,826451,668898,1.497385e+06,1.575829e+20,9.391186e+22,0.087737
2021-03-31,1947.27,1772.26,1841.40,574695.62,1.070069e+09,1919.08,1245144,599442,808391855,900640,726352,1.362753e+06,1.710658e+20,9.391186e+22,0.115229
2021-04-01,1982.01,1888.82,1919.08,452661.12,8.786759e+08,1967.67,1137569,547169,808939024,814582,652437,1.366159e+06,1.577857e+20,9.391186e+22,0.052073
